In [2]:
import pandas as pd
articleDetails = pd.read_pickle('article_details.pkl')

In [10]:
count = 0
for i in range(0, len(articleDetails['List Links'])):
    count += len(articleDetails['List Links'][i])
            
print("We have " + str(count) + " articles in our Corpus of data")

We have 3353 articles in our Corpus of data
